<a href="https://colab.research.google.com/github/Cehiim/TeoriaDosGrafos/blob/main/Projeto/grafos_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Informações gerais

## Integrantes
* Cesar Hideki Imai - 10402758
* João Victor Dallapé Madeira - 10400725
* David Varão Lima Bentes Pessoa - 10402647

# Setup

## Integração dos pacotes

O pacote `vectordb2` é usado para armazenar e recuperar textos usando técnicas de *chunking* (segmentação de texto), *embedding* (conversão de texto para vetores numéricos) e busca vetorial.

In [722]:
%pip install vectordb2

O pacote requests pode ser usado para recuperar o arquivo por meio de requisição em HTTP (esse pacote é opcional).

In [723]:
%pip install requests

O pacote `networkx` é usado para a criação, manipulação e representação de grafos.

In [724]:
%pip install networkx

Importação das bibliotecas

In [725]:
from vectordb import Memory
import requests
import networkx as nx
import matplotlib.pyplot as plt # Será usado para apresentação visual do grafo
import os # Será usado métodos para limpar o terminal para atualizar a interface em cada iteração do sistema
import time # Será usado método de espera para atualizar a interface gradualmente

## Classe Grafo

In [726]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 13 13:59:10 2023

@author: icalc
"""
class Grafo:
    TAM_MAX_DEFAULT = 100 # qtde de vértices máxima default
    # construtor da classe grafo
    def __init__(self, n=TAM_MAX_DEFAULT):
        self.n = n # número de vértices
        self.m = 0 # número de arestas
        # matriz de adjacência
        self.adj = [[0 for i in range(n)] for j in range(n)]

	# Insere uma aresta no Grafo tal que
	# v é adjacente a w
    def insereA(self, v, w):
        if self.adj[v][w] == 0:
            self.adj[v][w] = 1
            self.m+=1 # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] == 1:
            self.adj[v][w] = 0
            self.m -= 1  # atualiza qtd arestas

	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(f"Adj[{i:2d},{w:2d}] = 1 ", end="")
                else:
                    print(f"Adj[{i:2d},{w:2d}] = 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(" 1 ", end="")
                else:
                    print(" 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

## Classe GrafoR (Grafo direcionado rotulado)

In [727]:
# Grafo como uma matriz de adjacência rotulado
class GrafoR(Grafo):
# Não bota o init, vai bugar a classe

    def insereA(self, v, w, p):
        if self.adj[v][w] == 0:
            self.adj[v][w] = p
            self.m += 1  # atualiza qtd arestas

    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] != 0:
            self.adj[v][w] = 0
            self.m -= 1  # atualiza qtd arestas

    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f"Adj[{i:2d},{w:2d}] = {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                print(f" {self.adj[i][w]:.2f} ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )

    def insereV(self):
        for i in range(self.n):
            self.adj[i].append(0)
        self.n += 1
        self.adj.append([0]*self.n)

    def removeV(self, vertice):
        if(vertice >= self.n):
            return False
        for i in range(self.n-1):
            if(i >= vertice): # Substitui as conexões do vértice a ser retirado e
                              # os vértices posteriores a ele com as conexões do próximo vértice
                self.adj[i] = self.adj[i+1]
            self.removeA(i,vertice)
            self.adj[i].pop(vertice) # Remove o vértice escolhido da linha da matriz
        self.adj.pop() # Remove a última linha da matriz
        self.n -= 1
        return True

    def imprimeGrafo(self, vertices):
        # Criar um grafo dirigido usando a matriz de adjacência
        grafo = nx.DiGraph()  # Criar grafo direcionado

        # Adicionar vértices e arestas
        for i in range(self.n):
            grafo.add_node(i, label=vertices[i]["palavra"])
            for j in range(self.n):
                if self.adj[i][j] != 0:
                    peso_formatado = f"{self.adj[i][j]:.2f}"
                    grafo.add_edge(i, j, weight=peso_formatado)

        pos = nx.spring_layout(grafo, k=80000, scale=100000) # Layout para a posição dos nós

        # Desenhando os nós
        nx.draw_networkx_nodes(grafo, pos, node_size=6000, node_color='lightblue')

        # Desenhando as arestas
        nx.draw_networkx_edges(grafo, pos, edgelist=grafo.edges(), arrowstyle='-|>', arrowsize=8)

        # Desenhando os rótulos dos nós
        labels = nx.get_node_attributes(grafo, 'label')
        nx.draw_networkx_labels(grafo, pos, labels, font_size=8)

        # Desenhando os rótulos das arestas
        edge_labels = nx.get_edge_attributes(grafo, 'weight')
        nx.draw_networkx_edge_labels(grafo, pos, edge_labels=edge_labels)

        # Exibindo o gráfico
        plt.title(f"Grafo Direcionado Rotulado com {self.n} vértices e {self.m} arestas")
        plt.show()
'''
    def dfs_edges(self):
        visitados = [False] * len(self.vertices)
        edges = []

        def dfs(v):
            visitados[v] = True
            for i in range(len(self.vertices)):
                if self.matriz_adj[v][i] == 1 and not visitados[i]:
                    edges.append((v, i))
                    dfs(i)

        for nodo in range(len(self.vertices)):
            if not visitados[nodo]:
                dfs(nodo)

        return edges

    def tem_ciclo(self):
        visitados = [False] * len(self.vertices)
        rec_stack = [False] * len(self.vertices)

    def classificar_complexidade(grafo):
        if grafo.number_of_nodes() == 0:
            return 'C0'  # Grafo vazio

        if grafo.number_of_edges() == 0:
            return 'C1'  # Grafo com um único vértice ou sem arestas

        if grafo.tem_ciclo():
            return 'C3'  # Grafo tem ciclos

        if len(grafo.dfs_edges()) > 0:
            return 'C2'  # Grafo não tem ciclos, mas tem caminhos com mais de um vértice

        return 'C1'  # Grafo sem ciclos, todos os caminhos com exatamente um vértice

    def cfc(self):
        stack = []
        visitados = [False] * len(self.vertices)

        # Passo 1: Preencher a pilha com base na ordem de finalização do DFS
        for i in range(len(self.vertices)):
            if not visitados[i]:
                self.dfs_util(i, visitados, stack)

        # Passo 2: Criar o grafo transposto
        g_transposto = self.transpose()

        # Passo 3: Fazer DFS no grafo transposto de acordo com a pilha
        visitados = [False] * len(self.vertices)
        componentes = []

        while stack:
            v = stack.pop()
            if not visitados[v]:
                componente = []
                g_transposto.dfs_explorar(v, visitados, componente)
                componentes.append(componente)

        return componentes

    def grafo_reduzido(grafo):
        componentes = grafo.cfc()
        mapeamento = {}
        for i, componente in enumerate(componentes):
            for v in componente:
                mapeamento[v] = i

        matriz_reduzida = [[0 for _ in range(len(componentes))] for _ in range(len(componentes))]

        for u, v in grafo.arestas:
            if mapeamento[u] != mapeamento[v]:
                matriz_reduzida[mapeamento[u]][mapeamento[v]] = 1

        return matriz_reduzida
'''

"\n    def dfs_edges(self):\n        visitados = [False] * len(self.vertices)\n        edges = []\n\n        def dfs(v):\n            visitados[v] = True\n            for i in range(len(self.vertices)):\n                if self.matriz_adj[v][i] == 1 and not visitados[i]:\n                    edges.append((v, i))\n                    dfs(i)\n\n        for nodo in range(len(self.vertices)):\n            if not visitados[nodo]:\n                dfs(nodo)\n\n        return edges\n\n    def tem_ciclo(self):\n        visitados = [False] * len(self.vertices)\n        rec_stack = [False] * len(self.vertices)\n\n    def classificar_complexidade(grafo):\n        if grafo.number_of_nodes() == 0:\n            return 'C0'  # Grafo vazio\n\n        if grafo.number_of_edges() == 0:\n            return 'C1'  # Grafo com um único vértice ou sem arestas\n\n        if grafo.tem_ciclo():\n            return 'C3'  # Grafo tem ciclos\n\n        if len(grafo.dfs_edges()) > 0:\n            return 'C2'  # Graf

## Classe Memory

Aqui é utilizado a biblioteca VectorDB para criar uma memória virtual.

```
memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
```

- `chunking_strategy` define a estratégia de fragmentação dos dados. No modo "sliding_window", os dados são divididos em *chunks* (pedaços de texto) de tamanho fixo.

- `window_size` define a quantidade de palavras que um *chunk* representa. Neste caso, cada *chunk* representa uma palavra.

- `overlap` define quantos elementos de sobreposição existirão entre os *chunks* adjacentes. Neste caso, não haverá sobreposição já que as palavras usadas não formam frases, logo são independentes uma das outras.

### Método SAVE

O método é usado para fazer o *embedding* de uma palavra e inserir na memória.
```
memoria.save("palavra")
```
* `palavra` é a *string* da palavra desejada para inserir na memória.

### Método SEARCH

O método é usado para fazer o *embedding* de uma palavra para consultar a proximidade com as palavras salvas na memória através da busca vetorial.
```
busca = memoria.search(palavra, top_n=4)
```
* `palavra` é a string da palavra usada para consultar a proximidade com as palavras salvas na memória.
* `top_n` define o número de palavras retornadas da consulta.


### [Referência](https://vectordb.com/)

# Métodos

## 1. Ler dados

### Aquisição dos dados

In [728]:
def criaVertice(palavra, indice):
  vertice = {
      "palavra": palavra, # Recupera cada palavra e tira o "\n"
      "indice": indice,
      "proximos": []
  }
  return vertice

Os dados do documento são importados e guardados na variável `dados`.

In [729]:
def leArquivoHTTP():
  arquivo = requests.get('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/teste.txt').text

  lista = arquivo.split() # Distribui cada elemento do arquivo numa lista
  n_palavras = int(lista.pop(0)) # Separa o número de palavras (primeira linha do arquivo)
  vertices = []
  for i in range(n_palavras):
    vertice = criaVertice(lista[i], i)
    vertices.append(vertice)

  dados = [n_palavras]
  dados.append(vertices)

  return dados

In [730]:
def leArquivo(origem):
  try:
    with open(origem, 'r', encoding='utf-8') as arquivo:
      n_palavras = int(arquivo.readline()) # Recupera o número de palavras (primeira linha do arquivo)

      vertices = []
      for i in range(n_palavras):
        vertice = criaVertice(arquivo.readline().strip(), i)
        vertices.append(vertice)

    dados = [n_palavras]
    dados.append(vertices)

    return dados

  except FileNotFoundError:
      print("[Erro: Arquivo não encontrado]")

In [731]:
d = leArquivoHTTP()
#d = leArquivo("palavras.txt")
print(d)

[8, [{'palavra': 'Ecossistema', 'indice': 0, 'proximos': []}, {'palavra': 'Sustentabilidade', 'indice': 1, 'proximos': []}, {'palavra': 'Biodiversidade', 'indice': 2, 'proximos': []}, {'palavra': 'Reciclagem', 'indice': 3, 'proximos': []}, {'palavra': 'Conservação', 'indice': 4, 'proximos': []}, {'palavra': 'Poluição', 'indice': 5, 'proximos': []}, {'palavra': 'Desmatamento', 'indice': 6, 'proximos': []}, {'palavra': 'Reflorestamento', 'indice': 7, 'proximos': []}]]


### Embedding

Cada palavra é convertida para um vetor numérico e guardada na memória.

In [732]:
def embedding(memoria, n_palavras, vertices): # Método para fazer o embedding e inserção na memória de todas as palavras
  for i in range(n_palavras):
    memoria.save(vertices[i]["palavra"])

### Busca vetorial

Quanto menor é a distância, maior é a proximidade semântica.

In [733]:
def buscaVetorial(memoria, palavra): # Método para retornar os quatro elementos com maior proximidade semântica de uma palavra
  busca = memoria.search(palavra, top_n=4)
  return busca

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras três palavras mais próximas foram recuperadas as palavras de índice 1 até 4.

In [734]:
m = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0}) # Memória
n = d[0] # Número de palavras
v = d[1] # Lista de vértices (cada vértice é organizado em palavra, índice e vizinhos próximos)

embedding(m, n, v)

b = buscaVetorial(m, "Biodiversidade")
print(b)
print(f"\n\nBusca: Biodiversidade\n")
for i in range(1,4):
  palavra = b[i]['chunk']
  distancia = b[i]['distance']
  print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")

Initiliazing embeddings:  normal
OK.
[{'chunk': 'Biodiversidade', 'metadata': {}, 'distance': 0.0}, {'chunk': 'Sustentabilidade', 'metadata': {}, 'distance': 0.44970125}, {'chunk': 'Ecossistema', 'metadata': {}, 'distance': 0.5811522}, {'chunk': 'Poluição', 'metadata': {}, 'distance': 0.6769276}]


Busca: Biodiversidade

Palavra: Sustentabilidade
Distância: 0.45

Palavra: Ecossistema
Distância: 0.58

Palavra: Poluição
Distância: 0.68



### Integração no grafo

In [735]:
def buscaIndice(n_palavras, vertices, palavra):
  for i in range(n_palavras):
    if(vertices[i]["palavra"] == palavra):
      return vertices[i]["indice"]
  return -1

In [736]:
def buscaPalavra(n_palavras, vertices, indice):
  if(indice >= n_palavras or indice < 0):
    return "[Erro: índice inválido]"
  for i in range(n_palavras):
    if(vertices[i]["indice"] == indice):
      return vertices[i]["palavra"]
  return "[Erro: índice não encontrado]"

In [737]:
def criaVizinho(palavra, indice, distancia):
  proximo = {
      "vizinho": palavra,
      "indice": indice,
      "distancia": distancia
  }
  return proximo

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras três palavras mais próximas foram recuperadas as palavras de índice 1 até 4.

In [738]:
def integraGrafo(memoria, n_palavras, vertices):
  grafo = GrafoR(n_palavras) # Cria um grafo rotulado
  for i in range(n_palavras):
    busca = buscaVetorial(memoria, vertices[i]["palavra"])
    proximos = []
    for j in range(1,4):
      palavra = busca[j]['chunk']
      distancia = busca[j]['distance']
      proximo = criaVizinho(palavra, buscaIndice(n_palavras, vertices, palavra), distancia)
      proximos.append(proximo)
      grafo.insereA(vertices[i]["indice"], proximo["indice"], proximo["distancia"])
    vertices[i]["proximos"] = proximos
  return grafo

In [739]:
g = integraGrafo(m, n, v)
print(v[0]) # Informações do primeiro elemento da lista de vértices
g.showMin()

{'palavra': 'Ecossistema', 'indice': 0, 'proximos': [{'vizinho': 'Biodiversidade', 'indice': 2, 'distancia': 0.5811522}, {'vizinho': 'Sustentabilidade', 'indice': 1, 'distancia': 0.59003323}, {'vizinho': 'Reflorestamento', 'indice': 7, 'distancia': 0.7258596}]}

 n:  8 m: 24

 0.00  0.59  0.58  0.00  0.00  0.00  0.00  0.73 

 0.59  0.00  0.45  0.00  0.00  0.00  0.00  0.69 

 0.58  0.45  0.00  0.00  0.00  0.68  0.00  0.00 

 0.80  0.00  0.00  0.00  0.00  0.78  0.00  0.64 

 0.00  0.78  0.00  0.00  0.00  0.78  0.86  0.00 

 0.00  0.00  0.68  0.00  0.00  0.00  0.72  0.65 

 0.00  0.00  0.79  0.00  0.00  0.72  0.00  0.69 

 0.00  0.00  0.00  0.64  0.00  0.65  0.69  0.00 


fim da impressao do grafo.


## 2. Gravar dados

In [740]:
def gravaDados(n_palavras, vertices):
  with open("grafo.txt", "w") as arquivo:
    for i in range(n_palavras):
      palavra = vertices[i]["palavra"]
      arquivo.write(palavra+"\n")

In [741]:
gravaDados(n,v)

## 3. Inserir vértice

In [742]:
def insereVertice(grafo, n_palavras, vertices, palavra):
  if(buscaIndice(n_palavras, vertices, palavra) == -1):
    vertices.append(criaVertice(palavra, grafo.n))
    grafo.insereV()

In [744]:
insereVertice(g, n, v, "Maritaca")
print(v)
g.showMin()

[{'palavra': 'Ecossistema', 'indice': 0, 'proximos': [{'vizinho': 'Biodiversidade', 'indice': 2, 'distancia': 0.5811522}, {'vizinho': 'Sustentabilidade', 'indice': 1, 'distancia': 0.59003323}, {'vizinho': 'Reflorestamento', 'indice': 7, 'distancia': 0.7258596}]}, {'palavra': 'Sustentabilidade', 'indice': 1, 'proximos': [{'vizinho': 'Biodiversidade', 'indice': 2, 'distancia': 0.44970125}, {'vizinho': 'Ecossistema', 'indice': 0, 'distancia': 0.59003323}, {'vizinho': 'Reflorestamento', 'indice': 7, 'distancia': 0.692708}]}, {'palavra': 'Biodiversidade', 'indice': 2, 'proximos': [{'vizinho': 'Sustentabilidade', 'indice': 1, 'distancia': 0.44970125}, {'vizinho': 'Ecossistema', 'indice': 0, 'distancia': 0.5811522}, {'vizinho': 'Poluição', 'indice': 5, 'distancia': 0.6769276}]}, {'palavra': 'Reciclagem', 'indice': 3, 'proximos': [{'vizinho': 'Reflorestamento', 'indice': 7, 'distancia': 0.63607174}, {'vizinho': 'Poluição', 'indice': 5, 'distancia': 0.78495324}, {'vizinho': 'Ecossistema', 'indi

## 4. Inserir aresta

## 5. Remover vértice

## 6. Remover aresta

## 7. Exibir grafo

In [ ]:
g.imprimeGrafo(v)

## 8. Exibir matriz

In [ ]:
g.showMin()

## 9. Apresentar conexidade do grafo e o reduzido

# Menu

## Código

In [747]:
def menu():
    memoria = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})
    fim = False

    while(fim == False):
        print(
    '''
    Menu:
        1) Ler dados do arquivo
        2) Gravar dados no arquivo grafo.txt
        3) Inserir vértice
        4) Inserir aresta
        5) Remover vértice
        6) Remover aresta
        7) Exibir grafo
        8) Exibir matriz
        9) Apresentar a conexidade do grafo e o reduzido
        10) Encerrar a aplicação
    ''')
        choice = int(input())
        if choice == 1: # Cria grafo
            dados = leArquivoHTTP()
            #dados = leArquivo("palavras.txt")
            n_palavras = d[0]
            vertices = dados[1]
            embedding(memoria, n_palavras, vertices)
            grafo = integraGrafo(memoria, n_palavras, vertices)
            print("Grafo criado com sucesso!")

        elif choice == 2: # Grava dados no arquivo .txt
            try:
                gravaDados(n_palavras, vertices)
                print("Dados salvos com sucesso!")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 3: # Insere vértice
            print("Palavra a ser inserida: ")
            palavra = input()
            try:
                insereVertice(grafo, n_palavras, vertices, palavra)
                print("Vértice inserido com sucesso!")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 4: # Insere aresta
            try:
                print("Aresta inserida com sucesso!")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 5: # Remove vértice
            try:
                print("Vértice removido com sucesso!")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 6: # Remove aresta
            try:
                print("Aresta removida com sucesso!")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 7: # Exibe grafo
            try:
                grafo.imprimeGrafo(v)
                #ok = input("Aperte ENTER para continuar")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 8: # Exibe matriz
            try:
                grafo.showMin()
                ok = input("Aperte ENTER para continuar")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 9: # Apresenta a conexidade do grafo e grafo reduzido
            try:
                print("oi")
            except NameError:
                print("[Erro: Grafo não criado]")

        elif choice == 10: # Encerra
            fim = True
            print("Encerrando programa...")

        else:
            print("Opção inválida.")

        try:
            from IPython.display import clear_output

            time.sleep(2)
            clear_output(wait=True) # Limpa o terminal caso esteja no Google Colab

        except ImportError:
          time.sleep(2)
          if os.name == 'nt': # Limpa o terminal caso o OS seja Windows
              os.system('cls')
          else:
              os.system('clear') # Limpa o terminal caso o OS seja Linux ou MacOS

## Aplicação

In [748]:
menu()


    Menu:
        1) Ler dados do arquivo
        2) Gravar dados no arquivo grafo.txt
        3) Inserir vértice
        4) Inserir aresta
        5) Remover vértice
        6) Remover aresta
        7) Exibir grafo
        8) Exibir matriz
        9) Apresentar a conexidade do grafo e o reduzido
        10) Encerrar a aplicação
    
10
Encerrando programa...
